In [1]:
import sys
from math import floor, log10

In [2]:
# With operation count & significant airthmatic
def gauss_elimination_without_pivoting(a, b, d):
    '''
    Gaussian elimination without pivoting.
    param: a is an n x n matrix
           b is an n x 1 vector
           d is significant digit
    return: x is the solution of Ax=b.
            row echelon form of a
            row echelon form of b
            addition, subtraction, multiply, divide operations count
    '''
    n = len(a)
    x = [0 for i in range(n)]
    add_ops_count, mul_ops_count, div_ops_count = 0, 0, 0
    
    # Significant digit conversion
    def tidy(x, sig):
        y = abs(x)
        if y <= sys.float_info.min:
            return 0.0000
        return round(x, sig-int(floor(log10(y)))-1)
    
    # Apply forward elimination
    for i in range(n-1):
        # Check for leading element as non-zero
        if a[i][i] == 0:
            sys.exit("Triangle leading element zero detected, Division by Zero Error!")
        for j in range(i+1, n):
            multiplier = tidy(a[j][i] / a[i][i], d)
            div_ops_count = div_ops_count + 1
            # Apply row operation on matrix a
            a[j][i] = tidy(0, d)
            for col in range(i+1, n):
                a[j][col] = tidy(a[j][col] - tidy(multiplier * a[i][col], d), d)
                add_ops_count = add_ops_count + 1
                mul_ops_count = mul_ops_count + 1
            # Apply row operation on vector b
            b[j] = tidy(b[j] - tidy(multiplier * b[i], d), d)
            add_ops_count = add_ops_count + 1
            mul_ops_count = mul_ops_count + 1
    
    # Apply back substitution
    # Calculate rank of a to check if unique solution is present
    rank = 0
    zero_rows_idx = []
    for r_idx in range(n):
        if any(a[r_idx]):
            rank = rank + 1
        else:
            zero_rows_idx.append(r_idx)

    if rank == n:
        # System has one unique solution
        x[n-1] = tidy(b[n-1] / a[n-1][n-1], d)
        div_ops_count = div_ops_count + 1
        for i in range(n-2, -1, -1):
            x[i] = b[i]
            for j in range(i+1, n):
                x[i] = tidy(x[i] - tidy(a[i][j]*x[j], d), d)
                add_ops_count = add_ops_count + 1
                mul_ops_count = mul_ops_count + 1
            x[i] = tidy(x[i] / a[i][i], d)
            div_ops_count = div_ops_count + 1
        return {"ref_a": a, "ref_b": b, "solution": x, "add_ops_count": add_ops_count, 
                "mul_ops_count": mul_ops_count, "div_ops_count": div_ops_count}
    else:
        # r < n, check if r+1, r+2, ... r+n rows in b has any non zero value
        for z_idx in zero_rows_idx:
            if b[z_idx] != 0:
                sys.exit("Incosistent system, there is no solution!")
        sys.exit("Consistent system, there may be infinitly many solutions!")

In [3]:
# Test1
a = [[3.0,  2.0, -4.0],
     [2.0,  3.0,  3.0],
     [5.0, -3.0,  1.0]]
b = [3.0, 15.0, 14.0]
res = gauss_elimination_without_pivoting(a, b, 5)

print(f"REF of A: {res['ref_a']}\nREF of b: {res['ref_b']}\nSolution x: {res['solution']}")
print(f"No. of Addition Performed: {res['add_ops_count']}\nNo. of Multiplication Performed:: {res['mul_ops_count']}\nNo. of Division Performed:: {res['div_ops_count']}")

REF of A: [[3.0, 2.0, -4.0], [0.0, 1.6667, 5.6667], [0.0, 0.0, 29.2]]
REF of b: [3.0, 13.0, 58.4]
Solution x: [2.9999, 1.0002, 2.0]
No. of Addition Performed: 11
No. of Multiplication Performed:: 11
No. of Division Performed:: 6


In [4]:
# With operation count & significant airthmatic & partial pivoting
def gauss_elimination_with_pivoting(a, b, d):
    '''
    Gaussian elimination with partial pivoting.
    param: a is an n x n matrix
           b is an n x 1 vector
           d is significant digit
    return: x is the solution of Ax=b.
            row echelon form of a
            row echelon form of b
            addition, subtraction, multiply, divide operations count
    '''
    n =  len(a)
    x = [0 for i in range(n)]
    add_ops_count, mul_ops_count, div_ops_count = 0, 0, 0
    
    # Significant digit conversion
    def tidy(x, sig):
        y = abs(x)
        if y <= sys.float_info.min:
            return 0.0000
        return round(x, sig-int(floor(log10(y)))-1)
    
    # Apply forward elimination
    for i in range(n):
        max_idx = i
        max_val = a[max_idx][i]
        # Find the largest pivot element including i
        for j in range(i+1, n):
            if a[j][i] != 0.0 and abs(a[j][i]) > max_val:
                max_val, max_idx = a[j][i], j

        # Check if diagonal element is zero, which will cause divide by zero error
        if a[i][max_idx] == 0:
            if b[i] != 0:
                sys.exit("Singular matrix, Inconsistent system - no solutions!")
            else:
                sys.exit("Singular matrix, consistent system - may have infinitly many solutions!")
        
        # Swap the current row with larger value row
        if i != max_idx:
            for z in range(n):
                temp_a, temp_b = a[i][z], b[i]
                a[i][z], b[i] = a[max_idx][z], b[max_idx]
                a[max_idx][z], b[max_idx] = temp_a, temp_b
        for row in range(i+1, n):
            multiplier = tidy(a[row][i] / a[i][i], d)
            div_ops_count = div_ops_count + 1
            a[row][i] = tidy(0, d)
            for col in range(i+1, n):
                a[row][col] = tidy(a[row][col] - tidy(multiplier * a[i][col], d), d)
                mul_ops_count = mul_ops_count + 1
                add_ops_count = add_ops_count + 1
            b[row] = tidy(b[row] - tidy(multiplier * b[i], d), d)
            mul_ops_count = mul_ops_count + 1
            add_ops_count = add_ops_count + 1
            
    # Apply back substitution
    # Calculate rank of a to check if unique solution is present
    rank = 0
    zero_rows_idx = []
    for r_idx in range(n):
        if any(a[r_idx]):
            rank = rank + 1
        else:
            zero_rows_idx.append(r_idx)

    if rank == n:
        # System has one unique solution
        x[n-1] = tidy(b[n-1] / a[n-1][n-1], d)
        div_ops_count = div_ops_count + 1
        for i in range(n-2, -1, -1):
            x[i] = b[i]
            for j in range(i+1, n):
                x[i] = tidy(x[i] - tidy(a[i][j]*x[j], d), d)
                add_ops_count = add_ops_count + 1
                mul_ops_count = mul_ops_count + 1
            x[i] = tidy(x[i] / a[i][i], d)
            div_ops_count = div_ops_count + 1
        return {"ref_a": a, "ref_b": b, "solution": x, "add_ops_count": add_ops_count, 
                "mul_ops_count": mul_ops_count, "div_ops_count": div_ops_count}
    else:
        # r < n, check if r+1, r+2, ... r+n rows in b has any non zero value
        for z_idx in zero_rows_idx:
            if b[z_idx] != 0:
                sys.exit("Incosistent system, there is no solution!")
        sys.exit("Consistent system, there may be infinitly many solutions!")

In [5]:
# Test1
a = [[2.0,  2.0, 1.0],
     [4.0,  2.0, 3.0],
     [1.0, -1.0, 1.0]]
b = [6.0, 4.0, 0.0]
res = gauss_elimination_with_pivoting(a, b, 5)

print(f"REF of A: {res['ref_a']}\nREF of b: {res['ref_b']}\nSolution x: {res['solution']}")
print(f"No. of Addition Performed: {res['add_ops_count']}\nNo. of Multiplication Performed:: {res['mul_ops_count']}\nNo. of Division Performed:: {res['div_ops_count']}")

REF of A: [[4.0, 2.0, 3.0], [0.0, -1.5, 0.25], [0.0, 0.0, -0.33333]]
REF of b: [4.0, -1.0, 3.3333]
Solution x: [9.0, -1.0, -10.0]
No. of Addition Performed: 11
No. of Multiplication Performed:: 11
No. of Division Performed:: 6
